In [1]:
import sqlite3
import pandas as pd

In [2]:
# Connect to the marvel database
conn = sqlite3.connect('marvel.db')

# Get the table names
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query, conn)
table_names = tables["name"].tolist()

# Load the tables into DataFrames
if len(table_names) >= 2:
    stats_df = pd.read_sql(f"SELECT * FROM {table_names[0]} ORDER BY game_id DESC", conn)
    matches_df = pd.read_sql(f"SELECT * FROM {table_names[1]} ORDER BY game_id DESC", conn)

    # Display the DataFrames
    print(f"First table: {table_names[0]}")
    print(f"\nSecond table: {table_names[1]}")
else:
    print("Database contains less than two tables.")

# Close the connection
conn.close()

First table: stats

Second table: matches


In [3]:
matches_df.head()

,game_id,Map,Score,Replay ID,Match Length,Result,Game Mode
0,169,YGGDRASILL PATH,2:3,10461226090,14m 52s,LOSS,Competitive
1,168,ROYAL PALACE,1:2,10572222444,14m 53s,LOSS,Competitive
2,167,SYMBIOTIC SURFACE,1:2,10506427193,8m 0s,LOSS,Competitive
3,166,MIDTOWN,3:4,10976523234,19m 13s,LOSS,Competitive
4,165,HALL OF DJALIA,0:1,10622326105,6m 14s,LOSS,Competitive


In [4]:
# Extract Kills, Deaths, Assists from KDA
stats_df[['Kills', 'Deaths', 'Assists']] = stats_df['KDA'].str.extract(r'(\d+)\s*/\s*(\d+)\s*/\s*(\d+)')

# Convert columns to integers
stats_df[['Kills', 'Deaths', 'Assists']] = stats_df[['Kills', 'Deaths', 'Assists']].astype(int)

# Drop the KDA column
stats_df = stats_df.drop(columns=['KDA'])

# Define the new column order
new_column_order = [
    'game_id', 'Player', 'Rank', 'Kills', 'Deaths', 'Assists',
    'Damage', 'Dmg Taken', 'Healing', 'Heroes Played', 'Time Played'
]

# Reorder the DataFrame
stats_df = stats_df[new_column_order]

# Remove any non-numeric characters (like commas) and convert to numbers
numeric_columns = ['Damage', 'Dmg Taken', 'Healing']

for col in numeric_columns:
    stats_df[col] = stats_df[col].astype(str).str.replace(r'[^\d]', '', regex=True).astype(float)

In [5]:
# Ensure matches_df exists before proceeding
if 'matches_df' in locals() or 'matches_df' in globals():
    # Merge stats_df with matches_df to get game mode
    merged_df = stats_df.merge(matches_df[['game_id', 'Game Mode']], on='game_id', how='left')

    # Filter for Competitive games
    competitive_df = merged_df[merged_df['Game Mode'] == 'Competitive'].drop(columns=['Game Mode'])

else:
    print("Error: matches_df is not available. Ensure it is loaded before running this code.")

# Count how many Competitive games were played
num_games = competitive_df["game_id"].nunique()

# Display the result
print(f"Number of Competitive games: {num_games}")

Number of Competitive games: 158


In [6]:
# Define the target players
target_players = {"6macb", "TravyPG", "Camprin"}

# Find game_ids where all three players are present
game_counts = competitive_df.groupby("game_id")["Player"].apply(set)
valid_game_ids = game_counts[game_counts.apply(lambda players: target_players.issubset(players))].index

# Filter the DataFrame
filtered_df = competitive_df[competitive_df["game_id"].isin(valid_game_ids)]

# Count the number of unique game_id values
num_games = filtered_df["game_id"].nunique()

# Display the result
print(f"Number of games where 6macb, TravyPG, and Camprin were all present: {num_games}")

Number of games where 6macb, TravyPG, and Camprin were all present: 121


In [13]:
# Perform the join with matches_df on 'game_id'
full_df = filtered_df.merge(matches_df[['game_id', 'Map', 'Result', 'Game Mode']], 
                            on='game_id', how='left')

# Rename 'Result' to 'outcome' for consistency
full_df.rename(columns={'Result': 'outcome', 'Game Mode': 'game_mode'}, inplace=True)

# Identify 6macb's outcome and team placement in each game
game_team_map = full_df[full_df["Player"] == "6macb"][["game_id", "outcome"]].set_index("game_id")

# Determine if 6macb is in the top or bottom half
sixmacb_positions = full_df[full_df["Player"] == "6macb"].set_index("game_id")["Player"].to_dict()
full_df["team"] = full_df.groupby("game_id")["Player"].transform(
    lambda x: x.index < x.index[len(x) // 2]  # True for top half, False for bottom half
)

# Assign correct outcomes dynamically
def assign_outcome(row):
    game_id = row["game_id"]
    if game_id in game_team_map.index:
        sixmacb_outcome = game_team_map.loc[game_id, "outcome"]
        sixmacb_in_top_half = sixmacb_positions.get(game_id) in full_df[full_df["game_id"] == game_id]["Player"][:len(full_df[full_df["game_id"] == game_id]) // 2].values
        
        # Assign outcome based on 6macb's team
        if row["team"] == sixmacb_in_top_half:
            return sixmacb_outcome
        else:
            return "WIN" if sixmacb_outcome == "LOSS" else "LOSS"
    return row["outcome"]  # Default to existing value if 6macb is not in the game

full_df["outcome"] = full_df.apply(assign_outcome, axis=1)

# Drop temporary column
full_df.drop(columns=["team"], inplace=True)

In [15]:
# Function to filter heroes based on Time Played
def filter_heroes_by_time(row):
    heroes = row["Heroes Played"].split(" / ")
    times = row["Time Played"].split(" / ")

    # Convert times to numeric values (in seconds)
    time_seconds = []
    for t in times:
        if 'm' in t:
            time_seconds.append(int(t.replace('m', '')) * 60)
        elif 's' in t:
            time_seconds.append(int(t.replace('s', '')))
        else:
            time_seconds.append(0)  # Handle unexpected cases

    # Filter out heroes with less than 60 seconds played
    filtered_heroes = [heroes[i] for i in range(len(heroes)) if time_seconds[i] >= 60]
    filtered_times = [times[i] for i in range(len(times)) if time_seconds[i] >= 60]

    # Return updated strings
    return pd.Series([" / ".join(filtered_heroes), " / ".join(filtered_times)])

# Apply the filtering function to the DataFrame
full_df[['Heroes Played', 'Time Played']] = full_df.apply(filter_heroes_by_time, axis=1)

# Display the updated DataFrame
full_df.head(12)

,game_id,Player,Rank,Kills,Deaths,Assists,Damage,Dmg Taken,Healing,Heroes Played,Time Played,Map,outcome,game_mode
0,168,captt,Platinum 3,11,10,9,8495.0,15625.0,17734.0,Cloak and Dagger / Luna Snow,10m / 4m,ROYAL PALACE,WIN,Competitive
1,168,unfortunate000,Gold 2,21,8,1,17214.0,48844.0,0.0,Dr. Strange / Magneto,8m / 6m,ROYAL PALACE,WIN,Competitive
2,168,AgentStreusel,Gold 1,21,1,12,17341.0,4117.0,21115.0,Loki,14m,ROYAL PALACE,WIN,Competitive
3,168,SoopaWoopa,Gold 1,35,3,0,20495.0,6229.0,0.0,Star Lord,14m,ROYAL PALACE,WIN,Competitive
4,168,Edwinfast10,Platinum 3,23,10,9,16948.0,10895.0,0.0,Bucky,14m,ROYAL PALACE,WIN,Competitive
5,168,TheFloat,Gold 1,15,5,16,10568.0,13540.0,21231.0,Invisible Woman,14m,ROYAL PALACE,WIN,Competitive
6,168,SR405,Gold 1,11,6,24,2925.0,5200.0,31946.0,Rocket / Cloak and Dagger,9m / 5m,ROYAL PALACE,LOSS,Competitive
7,168,6macb,Gold 1,16,7,12,12654.0,25690.0,18883.0,Cloak and Dagger / Hulk,9m / 5m,ROYAL PALACE,LOSS,Competitive
8,168,TravyPG,Gold 1,12,10,0,15921.0,5628.0,0.0,Moon Knight / Iron Man,9m / 4m,ROYAL PALACE,LOSS,Competitive
9,168,Camprin,Gold 1,13,8,13,10071.0,19562.0,15845.0,Invisible Woman,14m,ROYAL PALACE,LOSS,Competitive


In [16]:
hero_roles = {
    "Hulk": "Tank",
    "Punisher": "DPS",
    "Storm": "DPS",
    "Loki": "Healer",
    "Dr. Strange": "Tank",
    "Mantis": "Healer",
    "Hawkeye": "DPS",
    "Captain America": "Tank",
    "Rocket": "Healer",
    "Hela": "DPS",
    "Cloak and Dagger": "Healer",
    "Black Panther": "DPS",
    "Groot": "Tank",
    "Majic": "DPS",
    "Moon Knight": "DPS",
    "Luna Snow": "Healer",
    "Squirrel Girl": "DPS",
    "Black Widow": "DPS",
    "Iron Man": "DPS",
    "Venom": "Tank",
    "Spiderman": "DPS",
    "Magneto": "Tank",
    "Scarlet Witch": "DPS",
    "Thor": "Tank",
    "Mr. Fantastic": "DPS",
    "Bucky": "DPS",
    "Penny": "Tank",
    "Star Lord": "DPS",
    "Namor": "DPS",
    "Adam Warlocke": "Healer",
    "Jeff": "Healer",
    "Psylocke": "DPS",
    "Wolverine": "DPS",
    "Invisible Woman": "Healer",
    "Iron Fist": "DPS"
}

In [17]:
# Create a new dataframe aggregating wins and total games per player per hero
winrate_df = full_df.copy()

# Count total games per player per hero
winrate_df["total_games"] = winrate_df.groupby(["Player", "Heroes Played"])["game_id"].transform("count")

# Count wins per player per hero
winrate_df["wins"] = winrate_df.apply(lambda row: 1 if row["outcome"] == "WIN" else 0, axis=1)
winrate_df["total_wins"] = winrate_df.groupby(["Player", "Heroes Played"])["wins"].transform("sum")

# Calculate win rate
winrate_df["winrate"] = (winrate_df["total_wins"] / winrate_df["total_games"]) * 100

# Select relevant columns and drop duplicates
winrate_df = winrate_df[["Player", "Heroes Played", "total_wins", "total_games", "winrate"]].drop_duplicates()


In [19]:
# Pivot the table so that each player has their own section with heroes as subcategories
winrate_pivot_df = winrate_df.pivot_table(index=["Player", "Heroes Played"], values=["total_wins", "total_games", "winrate"])

In [21]:
# Filter for only the selected players
filtered_winrate_df = winrate_pivot_df.loc[["6macb", "TravyPG", "Camprin"]]

In [24]:
filtered_winrate_df

total_games  total_wins  \
Player  Heroes Played                                                        
6macb   Cloak and Dagger                                    22          10   
        Cloak and Dagger / Hulk                              1           0   
        Cloak and Dagger / Luna Snow                         1           1   
        Cloak and Dagger / Penny                             1           0   
        Dr. Strange                                         34          12   
        Dr. Strange / Luna Snow                              1           0   
        Dr. Strange / Magneto                                1           0   
        Dr. Strange / Penny                                 11           5   
        Groot                                                4           2   
        Groot / Dr. Strange                                  2           0   
        Groot / Luna Snow                                    1           1   
        Groot / Magneto                                      1           0   
        Groot / Penny                                        2           1   
        Invisible Woman                                      6           3   
        Invisible Woman / Cloak and Dagger                   1           1   
        Luna Snow                                            8           5   
        Magneto                                              8           3   
        Magneto / Cloak and Dagger                           1           0   
        Magneto / Dr. Strange                                1           0   
        Magneto / Hulk                                       1           0   
        Mantis / Luna Snow                                   1           0   
        Mantis / Namor                                       1           0   
        Namor                                                1           0   
        Penny / Captain America / Dr. Strange                1           0   
        Penny / Dr. Strange                                  5           4   
        Rocket / Cloak and Dagger                            1           0   
        Star Lord / Dr. Strange                              1           0   
        Thor                                                 1           0   
        Thor / Dr. Strange                                   1           1   
TravyPG Black Widow / Squirrel Girl / Moon Knight            1           0   
        Bucky                                                1           0   
        Cloak and Dagger                                     1           0   
        Dr. Strange                                         11           9   
        Dr. Strange / Hulk / Wolverine / Namor               1           0   
        Dr. Strange / Psylocke / Thor                        1           0   
        Dr. Strange / Thor                                   3           0   
        Dr. Strange / Thor / Magneto                         1           0   
        Groot / Thor                                         3           2   
        Hela                                                 2           0   
        Hela / Hulk                                          1           0   
        Hela / Magneto                                       1           0   
        Hela / Moon Knight                                   1           0   
        Hela / Psylocke                                      1           0   
        Hulk                                                 3           0   
        Hulk / Moon Knight                                   1           0   
        Hulk / Thor                                          1           1   
        Luna Snow                                            2           0   
        Magneto                                              2           0   
        Magneto / Groot                                      1           0   
        Magneto / Psylocke                                   1           

In [25]:
# Function to split heroes and expand rows
def split_heroes(df):
    expanded_rows = []
    for _, row in df.iterrows():
        heroes = row.name[1].split(" / ")  # Splitting heroes from the 'Heroes Played' index
        for hero in heroes:
            expanded_rows.append([row.name[0], hero, row["total_games"], row["total_wins"], row["winrate"]])
    
    # Create new DataFrame with expanded rows
    new_df = pd.DataFrame(expanded_rows, columns=["Player", "Hero", "total_games", "total_wins", "winrate"])
    return new_df

# Apply the function to filter only selected players and expand heroes
expanded_winrate_df = split_heroes(filtered_winrate_df)

# Group by player and hero to sum up the games and wins per hero
expanded_winrate_df = expanded_winrate_df.groupby(["Player", "Hero"]).sum()

# Recalculate winrate after summing up wins and total games
expanded_winrate_df["winrate"] = (expanded_winrate_df["total_wins"] / expanded_winrate_df["total_games"]) * 100


In [29]:
# Sort the DataFrame by Player first, then by winrate in descending order
sorted_winrate_df = expanded_winrate_df.sort_values(by=["Player", "winrate"], ascending=[True, False])
sorted_winrate_df

total_games  total_wins     winrate
Player  Hero                                                 
6macb   Luna Snow                12.0         7.0   58.333333
        Invisible Woman           7.0         4.0   57.142857
        Penny                    20.0        10.0   50.000000
        Thor                      2.0         1.0   50.000000
        Cloak and Dagger         28.0        12.0   42.857143
        Groot                    10.0         4.0   40.000000
        Dr. Strange              58.0        22.0   37.931034
        Magneto                  13.0         3.0   23.076923
        Captain America           1.0         0.0    0.000000
        Hulk                      2.0         0.0    0.000000
        Mantis                    2.0         0.0    0.000000
        Namor                     2.0         0.0    0.000000
        Rocket                    1.0         0.0    0.000000
        Star Lord                 1.0         0.0    0.000000
Camprin Loki                      2.0         1.0   50.000000
        Scarlet Witch             6.0         3.0   50.000000
        Invisible Woman          42.0        17.0   40.476190
        Cloak and Dagger         73.0        29.0   39.726027
        Jeff                      7.0         2.0   28.571429
        Mr. Fantastic             5.0         1.0   20.000000
        Iron Fist                 1.0         0.0    0.000000
        Magneto                   1.0         0.0    0.000000
        Majic                     1.0         0.0    0.000000
        Penny                     3.0         0.0    0.000000
        Punisher                  1.0         0.0    0.000000
        Rocket                    3.0         0.0    0.000000
        Spiderman                 1.0         0.0    0.000000
TravyPG Venom                     3.0         3.0  100.000000
        Bucky                     4.0         3.0   75.000000
        Storm                     8.0         5.0   62.500000
        Dr. Strange              19.0        10.0   52.631579
        Psylocke                 37.0        16.0   43.243243
        Groot                    10.0         4.0   40.000000
        Namor                    20.0         8.0   40.000000
        Star Lord                 5.0         2.0   40.000000
        Thor                     37.0        14.0   37.837838
        Punisher                 10.0         3.0   30.000000
        Squirrel Girl             4.0         1.0   25.000000
        Hulk                      9.0         2.0   22.222222
        Moon Knight              17.0         3.0   17.647059
        Magneto                   7.0         1.0   14.285714
        Hela                      8.0         1.0   12.500000
        Black Widow               1.0         0.0    0.000000
        Cloak and Dagger          1.0         0.0    0.000000
        Hawkeye                   2.0         0.0    0.000000
        Iron Man                  1.0         0.0    0.000000
        Luna Snow                 2.0         0.0    0.000000
        Penny                     1.0         0.0    0.000000
        Wolverine                 1.0         0.0    0.000000